In [1]:
from uuid import uuid4
import os

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StdOutCallbackHandler
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from qdrant_client.models import PointStruct
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains import create_retrieval_chain
from langchain_qdrant import Qdrant

In [2]:
MODEL = "phi3:14b"
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

In [3]:
##https://arxiv.org/pdf/2404.18208
loader = PyPDFLoader("ros2.pdf")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
document = loader.load_and_split(text_splitter)

In [4]:
document[0]


Document(page_content='ROS 2 on aChip , Achieving Brain-Like Speeds and\nEfficiency in Robotic Networking\nVíctor Mayoral-Vilches, Juan Manuel Reina-Muñoz,\nMartiño Crespo Álvarez and David Mayoral-Vilches\nAcceleration Robotics\ncontact@accelerationrobotics.com\nAbstract\nThe Robot Operating System (ROS) pubsub model played a pivotal role in developing sophisticated\nrobotic applications. However, the complexities and real-time demands of modern robotics necessitate', metadata={'source': 'ros2.pdf', 'page': 0})

In [5]:
len(document)


87

In [6]:
from qdrant_client import QdrantClient
client = QdrantClient("localhost", port= 6333)

In [7]:
client.delete_collection(collection_name= "ros2")
#client.add(collection_name= "ros2", documents= document, ids= ids)

True

In [8]:
client.create_collection(
    collection_name="ros2",
    vectors_config={
        "content": VectorParams(size=5120, distance=Distance.COSINE)
        }
    )

True

In [9]:
def chunked_metadata(data, client=client, collection_name="ros2"):

    chunked_metadata = []

    for item in data:

      id = str(uuid4())
      content = item.page_content
      source = item.metadata["source"]
      page = item.metadata["page"]

      content_vector = embeddings.embed_documents([content])[0]
      vector_dict = {"content": content_vector}

      payload = {
           "page_content": content,
           "metadata": {
                        "id": id,
                        "page_content": content,
                        "source": source,
                        "page": page,
                        }
            }


      metadata = PointStruct(id=id, vector=vector_dict, payload=payload)
      chunked_metadata.append(metadata)

    client.upsert(
                  collection_name=collection_name,
                  wait=True,
                  points=chunked_metadata)

In [10]:
chunked_metadata(document[:10])


In [11]:
client.get_collections()


CollectionsResponse(collections=[CollectionDescription(name='ros2'), CollectionDescription(name='adidas_products'), CollectionDescription(name='customer_service'), CollectionDescription(name='sentences')])

In [12]:
ros_collection = client.get_collection("ros2")

print(f"Points: {ros_collection.points_count} ")

Points: 10 


In [13]:
client.search(
    collection_name="ros2",
    query_vector=("content", embeddings.embed_documents(["FPGA Platforms"])[0]),
    with_payload=["page_content", "source"],
    limit=5
)

[ScoredPoint(id='225a755f-6942-4997-8ce0-3ba8d03593d0', version=0, score=0.88135284, payload={'page_content': 'by more than 500 ×when compared to traditional ROS 2 software implementations on modern CPUs.\nAdditionally, it dramatically reduces maximum latency in ROS 2 networking communication by more\nthan 30,000 ×. In situations of peak latency, our design guarantees an isochronous response within 11\nmicroseconds, a stark improvement over the potential hundreds of milliseconds reported by modern\nCPU systems under similar conditions.\n1 Introduction'}, vector=None, shard_key=None),
 ScoredPoint(id='1f57d50c-326a-4312-9485-acc77cc5d2b4', version=0, score=0.86856097, payload={'page_content': 'ROS 2 on aChip , Achieving Brain-Like Speeds and\nEfficiency in Robotic Networking\nVíctor Mayoral-Vilches, Juan Manuel Reina-Muñoz,\nMartiño Crespo Álvarez and David Mayoral-Vilches\nAcceleration Robotics\ncontact@accelerationrobotics.com\nAbstract\nThe Robot Operating System (ROS) pubsub model p

In [14]:
vectorstore = Qdrant(client=client,
                     collection_name="ros2",
                     embeddings=embeddings,
                     vector_name="content")

In [15]:
template = """

You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \


Question: {input}
Context: {context}

Answer:

"""

retriever = vectorstore.as_retriever()

In [16]:
prompt = ChatPromptTemplate.from_template(template)
handler =  StdOutCallbackHandler()


In [17]:
combine_docs_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [18]:
result = retrieval_chain.invoke({"input": "how to use FPGA"})


In [19]:
result

{'input': 'how to use FPGA',
 'context': [Document(page_content='thousands-fold faster communications and better energy-efficiency, growing closer to the realm of hu-\nman brain-like speeds and efficiency.\nIn the fascinating realm of neuroscience, a single neuron firing an action potential in the human\nbrain lasts approximately 2 to 5 milliseconds, consuming an incredibly minute amount of energy —\nroughly 0.03 to 0.3 microjoules. These latencies and energy-efficiency set a benchmark that current', metadata={'id': '6ea093db-3bdb-453f-9ee9-a7bf37244d33', 'page': 0, 'page_content': 'thousands-fold faster communications and better energy-efficiency, growing closer to the realm of hu-\nman brain-like speeds and efficiency.\nIn the fascinating realm of neuroscience, a single neuron firing an action potential in the human\nbrain lasts approximately 2 to 5 milliseconds, consuming an incredibly minute amount of energy —\nroughly 0.03 to 0.3 microjoules. These latencies and energy-efficiency 

In [20]:
result["answer"]

' To use an FPGA (Field Programmable Gate Array) in the context of achieving brain-like speeds and efficiency in robotic networking as discussed in ROS 2 on aChip, you would follow these general steps:\n\n1. Identify your requirements for real-time communication in the robotic system that you aim to improve with an FPGA solution.\n\n2. Select an appropriate FPGA device based on factors like processing power, energy efficiency, and compatibility with existing hardware and software components of your project.\n\n3. Acquire a development board or chip for experimentation and implementation purposes; popular choices include Xilinx and Intel (formerly Altera) boards.\n\n4. Familiarize yourself with the FPGA programming environment and tools, such as Vivado Design Suite by Xilinx or Quartus Prime by Intel. These software suites help you in designing your custom logic circuits on the FPGA.\n\n5. Implement ROS 2 protocols for pubsub (publish-subscribe) communications using a high-level hardwar